# Exploratory Analysis

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import datetime

In [ ]:
# Read in Costs and Shipments
costsDf = pd.read_excel('data\Inv by Mfg.xlsx', sheet_name='Bin')
shipsDf = pd.read_excel('data\Sales by Mfg.xlsx', sheet_name='Invoiced Lines')

In [ ]:
print(costsDf.columns)
print()
print(costsDf.dtypes)
print()
print(shipsDf.columns)
print()
print(shipsDf.dtypes)

In [ ]:
# Remove the initial stocking of parts to get a better picture
# cypressCostsDf = cypressCostsDf.loc[cypressCostsDf['Created On'] > datetime.datetime(2018, 12, 30)]

In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Read in Costs and Shipments
costsDf = pd.read_excel('data\Inv by Mfg.xlsx', sheet_name='Bin')
shipsDf = pd.read_excel('data\Sales by Mfg.xlsx', sheet_name='Invoiced Lines')

values = []

# Create list of Mfgs sorted
mfgs = costsDf['Mfg'].unique().tolist()
for i in range(len(mfgs)):
    mfgs[i] = str(mfgs[i])
mfgs = sorted(mfgs)
mfgs = [m.upper() for m in mfgs]
mfgs = list(dict.fromkeys(mfgs))    # remove duplicates

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Flip Inventory"),
    dcc.Graph(id='graph'),
    html.Label([
        "MFG",
        dcc.Dropdown(
            id='mfgs-dropdown', clearable=False,
            value='CYPRESS SEMICONDUCTOR', options=[
                {'label': m, 'value': m}
                for m in mfgs
            ])
    ]),
    html.Label([
        "Type",
        dcc.Dropdown(
            id='type-dropdown', clearable=False,
            value='Total Cost by Day', options=[
                {'label': t, 'value': t}
                for t in ['Inventory Value', 'Total Cost by Day']
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("mfgs-dropdown", "value"),
     Input("type-dropdown", "value")]
)
def update_figure(mfg, typeStr):
    global costsDf
    global shipsDf
    global values
    
    # Get all purchases and shipments by mfg
    filteredCostsDf = costsDf.loc[costsDf['Mfg'].str.upper() == mfg.upper()]
    filteredShipsDf = shipsDf.loc[shipsDf['Mfg'].str.upper() == mfg.upper()]

    if typeStr == 'Total Cost by Day':

        filteredCostsDf = filteredCostsDf[['Bin Ext Value', 'Created On']]
    #     filteredShipsDf = filteredShipsDf[['Mfg', 'Qty', 'Resale', 'Invoice Date']]

        filteredCostsDf.columns = ['Cost', 'Date']
    #     filteredShipsDf.columns = ['Mfg', 'Resale Qty', 'Resale', 'Date']

        filteredCostsDf = filteredCostsDf.groupby('Date').agg({'Cost':'sum'}).reset_index()
    #     filteredShipsDf = filteredShipsDf.groupby('Invoice Date').agg({'Ext Resale':'sum'}).reset_index()

        return px.line(filteredCostsDf, x="Date", y="Cost",
                     title="Total Cost by Day")
    
    elif typeStr == 'Inventory Value':

        filteredCostsDf = filteredCostsDf[['Mfg', 'Bin Qty', 'Bin Cost', 'Created On']]
        filteredShipsDf = filteredShipsDf[['Mfg', 'Qty', 'Resale', 'Invoice Date']]

        filteredCostsDf.columns = ['Mfg', 'Cost Qty', 'Cost', 'Date']
        filteredShipsDf.columns = ['Mfg', 'Resale Qty', 'Resale', 'Date']

        costsAndShips = pd.concat([filteredCostsDf, filteredShipsDf], axis=0, ignore_index=True, sort=False)

        costsAndShips = costsAndShips.set_index("Date").groupby(pd.Grouper(freq='M')).agg({'Cost Qty':'sum', 'Resale Qty':'sum'})

        totInv = 0
        values = []
        for idx, row  in costsAndShips.iterrows():

            totInv += row['Cost Qty'] - row['Resale Qty']

            temp = totInv
            value = 0
            for idx2, row2 in filteredCostsDf.loc[filteredCostsDf.Date <= idx].iterrows():
                temp -= row2['Cost Qty']
                if temp <= 0:
                    value += row2['Cost'] * (-1) * temp
                    values.append(value)
                    break
                else:
                    value += row2['Cost'] * row2['Cost Qty']

        return px.line(x=range(len(values)), y=values, title='Inventory Value')
    
    else:
        return None

# Run app and display result inline in the notebook
app.run_server(mode='internal')
# app.run_server(mode='external')
# app.run_server(mode='jupyterlab')

# Dash App: Overview

In [5]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Read in data
#salesDf = pd.read_excel('C:\\Users\\marsh\\Documents\\work\\data\\June 2020\\Sales by Mfg 06-20.xlsx', sheet_name=None)
invDf = pd.read_excel('data\\Aged Inventory.xlsx', sheet_name='Stock')
#partsDf = pd.read_excel('data\\45292-parts.xlsx', sheet_name='Sheet1')

In [7]:
invDf.dtypes

PRC                           object
Part                          object
Receipt                        int64
Bin                           object
Primary Bin                  float64
Loc                           object
Bin Qty                        int64
Bin Cost                     float64
Bin Ext Value                float64
Shelf Qty                      int64
Created On            datetime64[ns]
Date Code                     object
Lot Code                      object
Receipt Date          datetime64[ns]
Written Off?                  object
Unnamed: 15           datetime64[ns]
Unnamed: 16                  float64
22444250.972611446           float64
dtype: object

In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Read in data
salesDf = pd.read_excel('C:\\Users\\marsh\\Documents\\work\\data\\June 2020\\Sales by Mfg 06-20.xlsx', sheet_name=None)
invDf = pd.read_excel('data\\Aged Inventory.xlsx', sheet_name='Stock')
partsDf = pd.read_excel('data\\45292-parts.xlsx', sheet_name='Sheet1')

# Merge parts info to inventory data
invDf["Prcpart"] = invDf["PRC"].astype(str) + invDf["Part"].astype(str)
mergedDf = invDf.merge(partsDf, how='left', on='Prcpart')
del invDf
del partsDf

# Create list of Dates for range slider
dateRange = mergedDf.copy()
# dateRange = sorted(dateRange['Receipt Date'].dt.to_period("M").unique().strftime('%m-%Y'))
dateRange = sorted(dateRange['Receipt Date'].dt.to_period("M").unique().to_timestamp())

# Create list of Mfgs sorted
mergedDf['Pref Manufacturer'] = mergedDf['Pref Manufacturer'].str.upper()
mfgs = mergedDf['Pref Manufacturer'].unique().tolist()
for i in range(len(mfgs)):
    mfgs[i] = str(mfgs[i])
mfgs = sorted(mfgs)
mfgs = list(dict.fromkeys(mfgs))    # remove duplicates

# Build App
app = JupyterDash(__name__, assets_url_path='assets')
app.layout = html.Div(
    [
        html.H1("Flip Dashboard"),
        html.Div(
            [
                html.Div(
                    [
                        html.P(
                            "Filter by top X MFGs (type value for X)",
                            className="control_label",
                        ),
                        dcc.Input(
                            id='mfgs-top',
                            placeholder='Enter a value...',
                            type='text',
                            value='10',
                            className='dcc_control'
                        ),
                        html.P(
                            "Select MFGs",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='mfgs-dropdown', clearable=False,
                            value=['CYPRESS SEMICONDUCTOR'], multi=True,
                            options=[
                                {'label': m, 'value': m}
                                for m in mfgs
                            ],
                            className='dcc_control'
                        ),
#                         html.P(
#                             "Select Date Range: ",
#                             className="control_label",
#                             id='output-container-range-slider'
#                         ),
#                         dcc.RangeSlider(
#                             id='date-slider',
#                             min=0,
# #                             max=len(dateRange) - 1,
#                             max=44,
# #                             step=None,
# #                             marks={k: v for k, v in enumerate(dateRange)},
# #                             value=[0, len(dateRange) - 1],
#                             value=[0, 44],
#                             updatemode='drag',
# #                             tooltip={'always_visible':False, 'placement':'topLeft'}
# #                             dots=False,
#                             className='dcc_control'
#                         ),
                        html.P(
                            "Select date range (or select range in graph)",
                            className="control_label",
                        ),
                        dcc.DatePickerRange(
                            id='date-picker-range',
                            min_date_allowed=dateRange[0],
                            max_date_allowed=dateRange[len(dateRange) - 1],
                            start_date=dateRange[0],
                            end_date=dateRange[len(dateRange) - 1],
                            display_format='MMM, YY',
                            className='dcc_control'
                        ),
                        html.P(
                            "Total aged inventory (in months)",
                            className="control_label",
                        ),
                        dcc.Input(
                            id='age-input',
                            placeholder='e.g. 6-12',
                            type='text',
                            value='12',
                            className='dcc_control'
                        ),
                    ],
                    id="cross-filter-options",
                    className="pretty_container four columns",
                ),
                html.Div(
                    [dcc.Graph(id="graph")],
#                     id="countGraphContainer",
#                     className="pretty_container",
                    id="right-column",
#                     className="eight columns",
                    className='pretty_container eight columns'
                ),
#                 html.Div(
#                     [
#                         html.Div(
#                             [dcc.Graph(id="graph")],
#                             id="countGraphContainer",
#                             className="pretty_container",
#                         ),
#                     ],
#                     id="right-column",
#                     className="eight columns",
#                 ),
            ],
            className="row flex-display",
        ),
        html.Div(
            [
                html.Div(
                    [dcc.Graph(id="graph2")],
                    className='pretty_container six columns'
                ),
                html.Div(
                    [dcc.Graph(id="graph3")],
                    className='pretty_container six columns'
                ),
            ],
            className="row flex-display",
        ),
        html.Div(
            [
                html.Div(
                    [dcc.Graph(id="graph4")],
                    className='pretty_container twelve columns'
                ),
            ],
            className="row flex-display",
        ),
    ],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"},
)

# Define callback to update mfg dropdown
@app.callback(
    Output('mfgs-dropdown', 'value'),
    [Input("mfgs-top", "value")]
)
def update_mfg_dropdown(top):
    if top != '':
        topMfgs = mergedDf.groupby('Pref Manufacturer').agg({'Bin Ext Value':'sum'}).reset_index().sort_values('Bin Ext Value', ascending=True).tail(int(top))['Pref Manufacturer']
        return topMfgs.tolist()
    else:
        return []

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("mfgs-dropdown", "value"),
     Input("age-input", "value"),
     Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date")]
)
def update_figure(mfg, age, startDate, endDate):
    
    # Total
    totalDf = mergedDf.copy()
    totalDf = totalDf[['Bin Ext Value', 'Receipt Date']]
    totalDf.columns = ['Value', 'Date']
    totalDf['Mfg'] = 'Total'
    totalDf.set_index("Date", inplace=True)
    totalDf.sort_values("Date", inplace=True)
    totalDf = totalDf.groupby([pd.Grouper(freq='M'), 'Mfg']).agg({'Value':'sum'}).cumsum().reset_index()
    
    # Mfg
    mfgDf = mergedDf.loc[mergedDf['Pref Manufacturer'].isin(mfg)]
    mfgDf = mfgDf[['Pref Manufacturer', 'Bin Ext Value', 'Receipt Date']]
    mfgDf.columns = ['Mfg', 'Value', 'Date']
    mfgDf.set_index("Date", inplace=True)
    mfgDf.sort_values("Date", inplace=True)
    mfgDf = mfgDf.groupby([pd.Grouper(freq='M'), 'Mfg']).agg({'Value':'sum'}).groupby('Mfg').cumsum().reset_index()
    
    # Aged
    agedDf = mergedDf.copy()
    agedDf = agedDf[['Bin Ext Value', 'Receipt Date']]
    agedDf.columns = ['Value', 'Date']
    agedDf['Mfg'] = 'Total Aged Inventory'
    agedDf.set_index("Date", inplace=True)
    agedDf.sort_values("Date", inplace=True)
    try:
        agedDf = agedDf.groupby([pd.Grouper(freq='M'), 'Mfg']).agg({'Value':'sum'}).shift(int(age), fill_value=0).cumsum().reset_index()
    except Exception as e:
        print(e)
    
    # Final
    finalDf = pd.concat([totalDf, agedDf, mfgDf], axis=0, ignore_index=True, sort=False)
    finalDf.sort_values('Date', inplace=True)
#     finalDf = finalDf.loc[(finalDf['Date'] > dateRange[date[0]]) & (finalDf['Date'] < dateRange[date[1]])]
    finalDf = finalDf.loc[(finalDf['Date'] > startDate) & (finalDf['Date'] < endDate)]

#     outDateStr = 'Select Date Range: ' + dateRange[date[0]].strftime('%B %Y') + ' to ' + dateRange[date[1]].strftime('%B %Y')

    return px.line(finalDf, x='Date', y='Value', color='Mfg', title='Inventory Trends')

# Define callback to update graph
@app.callback(
    Output('graph2', 'figure'),
    [Input("mfgs-dropdown", "value")]
)
def update_figure2(val):
    df = salesDf.copy()

    #Assign spreadsheet to a dataframe.
    il = df['Invoiced Lines'].loc[:, ['Customer', 'Invoice Date', 'Inside Sales','Prcpart','Qty','Cost','Ext Cost','Resale','Ext Resale', 'GM', 'Mfg']]
    il['Profit Margin'] = il['Ext Resale'] - il['Ext Cost']
    
    #Sum Profit Margin and create a new dataframe
    dff = il.groupby('Mfg').agg({'Profit Margin':'sum'}).reset_index()
    new_df = dff.sort_values(by='Profit Margin',ascending =False)

    #Bar graph of the top 10 manufacturers by profit margin
    return px.bar(new_df.head(10), x='Mfg' , y='Profit Margin', title='Top 10 Manufacturers by Profit Margin')

# Define callback to update graph
@app.callback(
    Output('graph3', 'figure'),
    [Input("mfgs-dropdown", "value")]
)
def update_figure3(val):
    df = salesDf.copy()
    
    #Assign spreadsheet to a dataframe.
    il = df['Invoiced Lines'].loc[:, ['Customer', 'Invoice Date', 'Inside Sales','Prcpart','Qty','Cost','Ext Cost','Resale','Ext Resale', 'GM', 'Mfg']]
    il['Profit Margin'] = il['Ext Resale'] - il['Ext Cost']
    
    #Sort Profit Margin by Inside Sales
    dfff = il.groupby('Inside Sales').agg({'Profit Margin':'sum'}).reset_index()
    new_dff = dfff.sort_values(by='Profit Margin',ascending =False)
    
    #Bar graph of top 5 Inside Sales by Profit Margin
    return px.bar(new_dff.head(), x='Inside Sales' , y='Profit Margin', title='Top 5 Inside Sales by Profit Margin')

# Define callback to update graph
@app.callback(
    Output('graph4', 'figure'),
    [Input("mfgs-dropdown", "value")]
)
def update_figure4(val):
    df = salesDf.copy()

    #Assign spreadsheet to a dataframe.
    il = df['Invoiced Lines'].loc[:, ['Customer', 'Invoice Date', 'Inside Sales','Prcpart','Qty','Cost','Ext Cost','Resale','Ext Resale', 'GM', 'Mfg']]
    il['Profit Margin'] = il['Ext Resale'] - il['Ext Cost']
    
    #Sum Profit Margin and create a new dataframe
    dataframe = il.groupby('Invoice Date').agg({'Profit Margin':'sum'}).reset_index()
    new_dataframe = dataframe.sort_values(by='Invoice Date',ascending =True)

    #Plotting line graph of Profit Margin characterized by Invoice Date.
    return px.line(new_dataframe, x='Invoice Date' , y='Profit Margin', title='Daily Total Profit Margin')

# Run app and display result inline in the notebook
# app.run_server(mode='inline')
# app.run_server(mode='jupyterlab')
app.run_server(mode='external')


Dash app running on http://127.0.0.1:8050/


# Dash App: By Mfg

In [2]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Read in data
invDf = pd.read_excel('data\\Aged Inventory.xlsx', sheet_name='Stock')
partsDf = pd.read_excel('data\\45292-parts.xlsx', sheet_name='Sheet1')
salesDf = pd.read_excel('C:\\Users\\marsh\\Documents\\work\\data\\June 2020\\Sales by Mfg 06-20.xlsx', sheet_name=None)

# Merge parts info to inventory data
invDf["Prcpart"] = invDf["PRC"].astype(str) + invDf["Part"].astype(str)
mergedDf = invDf.merge(partsDf, how='left', on='Prcpart')
del invDf
del partsDf

# Create list of Dates for range slider
dateRange = mergedDf.copy()
dateRange = sorted(dateRange['Receipt Date'].dt.to_period("M").unique().to_timestamp())

# Create list of Mfgs sorted
mergedDf['Pref Manufacturer'] = mergedDf['Pref Manufacturer'].str.upper()
mfgs = mergedDf['Pref Manufacturer'].unique().tolist()
for i in range(len(mfgs)):
    mfgs[i] = str(mfgs[i])
mfgs = sorted(mfgs)
mfgs = list(dict.fromkeys(mfgs))    # remove duplicates

# Build App
app = JupyterDash(__name__, assets_url_path='assets')
app.layout = html.Div(
    [
        html.H1("Manufacturer Dashboard"),
        html.Div(
            [
                html.Div(
                    [
                        html.P(
                            "Select MFG",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='mfgs-dropdown', clearable=False,
                            value='CYPRESS SEMICONDUCTOR',
                            options=[
                                {'label': m, 'value': m}
                                for m in mfgs
                            ],
                            className='dcc_control'
                        ),
                        html.P(
                            "Select date range (or select range in graph)",
                            className="control_label",
                        ),
                        dcc.DatePickerRange(
                            id='date-picker-range',
                            min_date_allowed=dateRange[0],
                            max_date_allowed=dateRange[len(dateRange) - 1],
                            start_date=dateRange[0],
                            end_date=dateRange[len(dateRange) - 1],
                            display_format='MMM, YY',
                            className='dcc_control'
                        ),
                        html.P(
                            "Age of inventory (in months)",
                            className="control_label",
                        ),
                        dcc.Input(
                            id='age-input',
                            placeholder='e.g. 6-12',
                            type='text',
                            value='12',
                            className='dcc_control'
                        ),
                    ],
                    id="cross-filter-options",
                    className="pretty_container four columns",
                ),
                html.Div(
                    [dcc.Graph(id="graph")],
                    id="right-column",
                    className='pretty_container eight columns'
                ),
            ],
            className="row flex-display",
        ),
        html.Div(
            [
                html.Div(
                    [dcc.Graph(id="graph4")],
                    className='pretty_container twelve columns'
                ),
            ],
            className="row flex-display",
        ),
    ],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"},
)

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("mfgs-dropdown", "value"),
     Input("age-input", "value"),
     Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date")]
)
def update_figure(mfg, age, startDate, endDate):
    
    # Mfg
    mfgDf = mergedDf.loc[mergedDf['Pref Manufacturer'] == mfg]
    mfgDf = mfgDf[['Bin Ext Value', 'Receipt Date']]
    mfgDf.columns = ['Value', 'Date']
    mfgDf['Type'] = 'Total Inventory'
    mfgDf.set_index("Date", inplace=True)
    mfgDf.sort_values("Date", inplace=True)
    mfgDf = mfgDf.groupby([pd.Grouper(freq='M'), 'Type']).agg({'Value':'sum'}).groupby('Type').cumsum().reset_index()
    
    # Aged
    agedDf = mergedDf.loc[mergedDf['Pref Manufacturer'] == mfg]
    agedDf = agedDf[['Bin Ext Value', 'Receipt Date']]
    agedDf.columns = ['Value', 'Date']
    agedDf['Type'] = 'Aged Inventory' + ' (' + str(age) + ' months)'
    agedDf.set_index("Date", inplace=True)
    agedDf.sort_values("Date", inplace=True)
    try:
        agedDf = agedDf.groupby([pd.Grouper(freq='M'), 'Type']).agg({'Value':'sum'}).shift(int(age), fill_value=0).cumsum().reset_index()
    except Exception as e:
        print(e)
    
    # Final
    finalDf = pd.concat([agedDf, mfgDf], axis=0, ignore_index=True, sort=False)
    finalDf.sort_values('Date', inplace=True)
    finalDf = finalDf.loc[(finalDf['Date'] > startDate) & (finalDf['Date'] < endDate)]

    return px.line(finalDf, x='Date', y='Value', color='Type', title='Inventory Trends')

# Define callback to update graph
@app.callback(
    Output('graph4', 'figure'),
    [Input("mfgs-dropdown", "value")]
)
def update_figure4(mfg):
    df = salesDf['Invoiced Lines']
    df = df.loc[df['Mfg'] == mfg]

    #Assign spreadsheet to a dataframe.
    il = df.loc[:, ['Customer', 'Invoice Date', 'Inside Sales','Prcpart','Qty','Cost','Ext Cost','Resale','Ext Resale', 'GM', 'Mfg']]
    il['Profit Margin'] = il['Ext Resale'] - il['Ext Cost']
    
    #Sum Profit Margin and create a new dataframe
    dataframe = il.groupby('Invoice Date').agg({'Profit Margin':'sum'}).reset_index()
    new_dataframe = dataframe.sort_values(by='Invoice Date',ascending =True)

    #Plotting line graph of Profit Margin characterized by Invoice Date.
    return px.line(new_dataframe, x='Invoice Date' , y='Profit Margin', title='Daily Profit Margin')

# Run app and display result inline in the notebook
# app.run_server(mode='inline')
# app.run_server(mode='jupyterlab')
app.run_server(mode='external')


Dash app running on http://127.0.0.1:8050/
